In [1]:
#pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
#pip install transformers[sentencepiece]

In [3]:
MODEL = 'EMBEDDIA/litlat-bert'

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from nltk.tokenize import sent_tokenize
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

## Load tokenizer and model

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL) 
model = AutoModelForMaskedLM.from_pretrained(MODEL) 
 


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [6]:
# https://www.kaggle.com/vbookshelf/basics-of-bert-and-xlm-roberta-pytorch
# https://anubhav20057.medium.com/step-by-step-guide-abstractive-text-summarization-using-roberta-e93978234a90

Check vocab size

In [7]:
tokenizer.vocab_size

84196

What are the special tokens

In [8]:
tokenizer.special_tokens_map



{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [9]:
print('bos_token_id <s>:', tokenizer.bos_token_id)
print('eos_token_id </s>:', tokenizer.eos_token_id)
print('sep_token_id </s>:', tokenizer.sep_token_id)
print('pad_token_id <pad>:', tokenizer.pad_token_id)


bos_token_id <s>: 0
eos_token_id </s>: 2
sep_token_id </s>: 2
pad_token_id <pad>: 84196


### How to use tokenizer to create XLM-RoBERTa input

In [10]:
MAX_LEN = 15

sentence1 = 'Laba diena.'
sentence2 = 'Kaip sekasi?'

encoded_dict = tokenizer.encode_plus(
            sentence1, sentence2,      
            add_special_tokens = True,
            truncation= True,
            max_length = MAX_LEN,     
            pad_to_max_length = True,
            return_attention_mask = True,   
            return_tensors = 'pt' # return pytorch tensors
       )


encoded_dict

C:\Users\user2\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2137: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': tensor([[    0, 22716,  2478,    14,     2,     2,  1021, 24924,  7283,     2,
         84196, 84196, 84196, 84196, 84196]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}

In [11]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

# These are torch tensors.
print(input_ids)
print(att_mask)

tensor([    0, 22716,  2478,    14,     2,     2,  1021, 24924,  7283,     2,
        84196, 84196, 84196, 84196, 84196])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])


### Decoding a sequence of tokens

In [12]:
input_ids = encoded_dict['input_ids'][0]

print(input_ids)

tensor([    0, 22716,  2478,    14,     2,     2,  1021, 24924,  7283,     2,
        84196, 84196, 84196, 84196, 84196])


In [13]:
# skip_special_tokens – if set to True, will replace special tokens.

a = tokenizer.decode(input_ids,
                skip_special_tokens=False)

b = tokenizer.decode(input_ids,
                skip_special_tokens=True)



print(a)
print(b)

<s> Laba diena.</s></s> Kaip sekasi?</s><pad><pad><pad><pad><pad>
Laba diena. Kaip sekasi?


### Overflowing tokens and Stride
When a sentence is truncated (because it's length exceeds max_length) it's possible to get the tokenizer to return the tokens that were cut off. These truncated tokens will be returned in a list called overflowing_tokens.

In [14]:
MAX_LEN = 15 # This value could be set as 256, 512 etc.

sentence1 = 'Laba diena. Kaip jums sekasi?'
sentence2 = 'šiandien labai grazi diena ir cia dar ne viskas!'


encoded_dict = tokenizer.encode_plus(
            sentence1, sentence2,
            add_special_tokens=True,
            #truncation = True
            max_length = MAX_LEN,
            stride=0,
            padding = 'max_length',
            return_tensors = 'pt',
            return_overflowing_tokens=True,
       )


encoded_dict

{'input_ids': tensor([[    0, 22716,  2478,    14,  1021,  1411, 24924,  7283,     2,     2,
           750,   273,  6317,   303,  4917,  4075,  2478,    11,  5311,   190,
            77,  1860,  7821,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'overflow_to_sample_mapping': tensor([0])}

***
## Tokenize the data

In [362]:
import pandas as pd

d = {'Text' : [min_input, delfi_input]}
df = pd.DataFrame(data=d)

df

,Text
0,Susitelkti teko visiems Praeitos savaitės paba...
1,Statant tiltą ledonešio rizikos nenumatė Ar pa...


In [363]:
#!pip install datasets
from datasets import Dataset

In [364]:
train_data = Dataset.from_pandas(df)
train_data


Dataset({
    features: ['Text'],
    num_rows: 2
})

In [365]:
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

#parameter setting
batch_size=256  #
encoder_max_length=40
decoder_max_length=8

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  #outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  #batch["decoder_input_ids"] = outputs.input_ids
  #batch["decoder_attention_mask"] = outputs.attention_mask
  #batch["labels"] = outputs.input_ids.copy()

  # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  #batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text"]#, "Summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask"]#, "decoder_input_ids", "decoder_attention_mask", "labels"],
)

## processing validation data
#val_data = val_data.map(
#    process_data_to_model_inputs, 
#    batched=True, 
#    batch_size=batch_size, 
#    remove_columns=["Text", "Summary"]
#)
#val_data.set_format(
#    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
#)

In [366]:
train_data['input_ids']

tensor([[    0,  3116,   203, 11863,   140,  4346,  2541,  5286,  6883,    73,
         13423,  5283, 38387,   429, 16741,   111,   228,   598, 23218,  4347,
          1702,  1054,    56, 32942, 27391,  5167,  2032,  2098,    33,  7133,
           373,    42,  9243,  2650,  1586,   103, 11259,  2098,   103,     2],
        [    0,  8016,   210, 10475,  1349,  1137,   627,   501,   174,  7264,
         38699, 18905,   331,  6753, 17816,   449, 23973,  7772,   600,  8376,
           136,  2098,   225,  5503,   951,  2333,  1782, 38929,  1060, 20438,
          4944, 22300, 34505, 11967,   194,  2854,   641,    97,  4716,     2]])

In [31]:
inputs = tokenizer("Laba diena!", return_tensors="pt")
outputs = model(**inputs)

In [32]:
inputs

{'input_ids': tensor([[    0, 22716,  2478,  7821,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

***

In [367]:
def bertSent_embeding(sentences):
    """
    Input a list of sentence tokens
    
    Output a list of latent vectors, each vector is a sentence representation
    
    Note: Bert model produce 12 layers of latent vector, the 'last layer' method is used here,
          other choices includes average last 4 layers, average all layers, etc.
    
    """
    ## Add sentence head and tail as BERT requested
    #marked_sent = ["[CLS] " +item + " [SEP]" for item in sentences] # skiriasi rezultatai su situ dalyku ir be
    
    ## USE Bert tokenizization 
    #tokenizer = BertTokenizer.from_pretrained(MODEL)
    tokenized_sent = [tokenizer.tokenize(item) for item in sentences]#marked_sent]

    ## index to BERT vocabulary
    indexed_tokens = [tokenizer.convert_tokens_to_ids(item) for item in tokenized_sent]
    #print("indexed tokens: ",indexed_tokens[0])
    tokens_tensor = [torch.tensor([item]) for item in indexed_tokens]
    #print("tokens tensors: ",tokens_tensor[0])
    ## add segment id as BERT requested
    segments_ids = [[1] * len(item) for ind,item in enumerate(tokenized_sent)]
    segments_tensors = [torch.tensor([item]) for item in segments_ids]
    
    #print(segments_tensors,'\n\n-------------\n', tokens_tensor)
    
    ## load BERT base model and set to evaluation mode
    bert_model = model#BertModel.from_pretrained('bert-base-uncased')
    bert_model.eval()
    
    ## Output 12 layers of latent vector
    assert len(tokens_tensor) == len(segments_tensors)
    encoded_layers_list = []
    for i in range(len(tokens_tensor)):
        with torch.no_grad():
            encoded_layers = bert_model(tokens_tensor[i], segments_tensors[i])
        encoded_layers_list.append(encoded_layers)
        


    ## Use only the last layer vetcor, other choice available
    token_vecs_list = [layers.logits[0] for layers in encoded_layers_list] # layers[11][0] reik patikslinti kas cia vyksta
    
    ## Pooling word vector to sentence vector, use mean pooling, other choice available
    sentence_embedding_list = [torch.mean(vec, dim=0).numpy() for vec in token_vecs_list]
    
    
    
    return sentence_embedding_list




def kmeans_sumIndex(sentence_embedding_list):
    """
    Input a list of embeded sentence vectors
    
    Output an list of indices of sentence in the paragraph, represent the clustering of key sentences
    
    Note: Kmeans is used here for clustering
    
    """
    n_clusters = np.ceil(len(sentence_embedding_list)**0.5)
    kmeans = KMeans(n_clusters=int(n_clusters))
    kmeans = kmeans.fit(sentence_embedding_list)
    
    sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
    
    sum_index = sorted(sum_index)
    
    return sum_index


In [368]:

def bertSummarize(text):
    """
    Input a paragraph as string
    
    Output the summary including a few key sentences using BERT sentence embedding and clustering
    """
    sentences = sent_tokenize(text)
    
    sentence_embedding_list = bertSent_embeding(sentences)
    sum_index = kmeans_sumIndex(sentence_embedding_list)
    
    summary = ' '.join([sentences[ind] for ind in sum_index])
    
    return summary

In [370]:
# viso teksto skaidymas i sakinius
sentence_tokenize = sent_tokenize(delfi_input)
sentence_tokenize

['Statant tiltą ledonešio rizikos nenumatė Ar pasikartos prieš dešimtmetį įvykusi nelaimė, kai Neryje ties Kauno stichiškai susiformavus ledo lyčių sangrūdoms buvo užsemtas Radikių kaimas Kaune rajone?',
 'Ekspertams prognozuojant, kad šįkart potvynis gali vėl pasikartoti, Kauno rajono savivaldybė jau ruošiasi galimam stichijos šėlsmui.',
 'Ypač daug nerimo kelia tai, kad statant naująjį tiltą per Nerį, ženkliai susiaurinta upės vaga.',
 'Link Kauno tekančios Neries vanduo šioje vietoje patenka tarsi į piltuvą.',
 'Upė šiuo metu vis dar užšalusi, gali būti, jog ledai pajudės šios savaitės pabaigoje, ar kitą savaitę.Praėjusią savaitę Kauno rajono savivaldybėje surengtame Ekstremaliųjų situacijų operacijų centro posėdyje aiškintasi, ką reikėtų daryti, jeigu susiformuotų ledų sangrūdos ir grėstų potvynis.',
 '„Ledo lytys kai kuriose Neries vietose jau yra sustojusios ir tai kelia nerimą.',
 'Nesinorėtų, kad pasikartotų prieš 10 metų įvykusi situacija“, – sakė Kauno rajono savivaldybės adm

### Santrauka

In [371]:
test_inp = 'Rankų higiena – pagrindinė profilaktikos priemonė, mažinanti rota virusinės infekcijos plitimo riziką. Plaunant rankas su muilu, nuo jų pašalinama beveik 95 proc. žmogui patogeninių bakterijų ir virusų. Rankas plauti būtina ne tik pasinaudojus tualetu ar po sąlyčio su ligonio išmatomis, bet grįžus  iš parduotuvės, po važiavimo visuomeniniu transportu, kiekvieną kartą prieš valgį ir maisto gaminimą. Kol maži vaikai nesugeba patys nusiplauti tinkamai rankų, tai turėtų daryti kolektyvuose personalas, o namuose – tėvai; Kadangi ikimokyklinio amžiaus vaikai labai imlūs rotavirusinei infekcijai, todėl ji lengvai  plinta kolektyvuose. Jei vaikas suviduriavo, apsivėmė, patariama tokio vaiko neleisti į darželį ar kitus vaikų susibūrimus. Sergantysis virusus išskiria vidutiniškai 7 dienas, todėl vaiko į kolektyvą nereikėtų leisti, kol yra klinikiniai ligos simptomai ir dar kelias dienas; Ligoniui namuose reikėtų paskirti atskirą kambarį ar jo dalį. Kambarį, kuriame guli ligonis, būtina gerai vėdinti. Sergančiojo slaugai naudoti kuo daugiau vienkartinių slaugos priemonių; Tualetą ir ligonio kambarį valyti atskirais įrankiais, šluostėmis, naudojant chloro turinčias dezinfekcines medžiagas; Saugiai šalinti sergančiojo kūdikio sauskelnes, atskirai laikyti ir skalbti nešvarius ligonio rankšluosčius, drabužius. Patartina juos 15 minučių pavirinti su skalbimo milteliais. Išdžiovintus skalbinius gerai išlyginti karšta laidyne; Vaiko žaislus plauti muilo ar sodos tirpalu, skalauti tekančiu vandeniu, išdžiovinti; Gydytojai rekomenduoja skiepyti vaiką nuo rota viruso. Tai sumažins jo tikimybę ir, jei jis tai įgis, simptomai bus ne tokie sunkūs.'
test_inp2 = 'Specifinio vaisto rota virusui gydyti nėra. Antibiotikais ar antivirusiniai vaistais tokio ligonio gydyti negalima. Bene svarbiausias gydymo būdas – per dieną išgerti daug skysčių. Kiekvienam suaugusiam ligoniui patariama išgerti apie 2,5-3 litrus vandens. Litru mažiau skysčių per dieną turėtų suvartoti vaikai. Žinoma, ne ką mažiau svarbi yra ir mityba. Roto viruso metu žmogui reikėtų valgyti tik lengvai virškinamą maistą, nenaudoti daug prieskonių ar druskų. Taip pat kartais žmonės daro klaidą gerdami daug sulčių. Sergant roto virusu, sulčių, ypač šviežiai spaustų, nederėtų gerti. Gydytojas gali pasiūlyti vaistą simptomams palengvinti. Paklauskite skysčių, skirtų rehidratacijai, kad pakeistumėte mineralus, prarastus dėl vėmimo ir viduriavimo. Norint greičiau pasveikti, patariama vartoti probiotikus. Kadangi roto virusas uždegimą sukelia tik prikibęs prie plonųjų žarnų sienelių, tai probiotikų – gerųjų bakterijų – teikiama apsauga virškinamajam traktui yra naudinga. Vartodami Lactobacillus GG galime ne tik apsisaugoti nuo virusų, prilipusių prie žarnyno sienelių, bet ir, probiotikų pagalba sukūrę daugiau apsauginių antikūnų, sunaikinti roto virusą, kol jis nepadarė jokios žalos.'
test_inp3 = 'didžiųjų skaičių dėsnis, dėsnis teigia, kad tam tikromis sąlygomis atsitiktinių veiksnių visuma ilgainiui nulemia rezultatą, nepriklausantį arba mažai priklausantį nuo atsitiktinumo. Tikimybių teorijoje ir matematinėje statistikoje didžiųjų skaičių dėsnis teigia, kad vienodomis sąlygomis daug kartų stebint tą patį atsitiktinį dydį stebėtų reikšmių aritmetinis vidurkis artėja prie tam tikros neatsitiktinės reikšmės, vadinamos stebimojo dydžio (teoriniu) vidurkiu. Kai vienodomis sąlygomis ilgą laiką stebimas tas pats įvykis, kurio pasirodymo tikimybė yra p, didžiųjų skaičių dėsnis teigia, kad to įvykio pasirodymų santykinis dažnis artėja prie p. Pvz., neribotai metant simetrišką monetą santykinis herbo atvirtimų dažnis artėja prie teorinės tokio atvirtimo tikimybės, lygios. Vienas didžiųjų skaičių dėsnio matematinių tikslių formulavimų yra toks: jei X1, X2, …, Xn, …, yra nepriklausomųjų vienodai pasiskirsčiusių atsitiktinių dydžių su vidurkiais EXn = a seka, tai aritmetinis vidurkis (X1 + X2 + ... + Xn)/n artėja prie a, kai n neribotai auga. Pagal artėjimo pobūdį (su tikimybe 1 arba pagal tikimybę) skiriama stiprusis arba silpnasis didžiųjų skaičių dėsnis. Dėsnio apibendrinimai nagrinėja atvejus, kai atsitiktiniai dydžiai nėra vienodai pasiskirstę arba nėra nepriklausomi. Nepriklausomųjų atsitiktinių dydžių sumų ribinę elgseną tiksliau aprašo centrinė ribinė teorema. Atskirą didžiųjų skaičių dėsnio atvejį suformulavo ir 1713 paskelbė J. Bernoullis. 1837 apibendrino ir didžiųjų skaičių dėsnio terminą pirmasis pavartojo S. D. Poissonas. 1867 didžiųjų skaičių dėsnio sąvoką praplėtė P. Čebyšovas.'
test_inp4 = 'atsitiktinių procesų teorija, tikimybių teorijos šaka, tirianti atsitiktinius procesus, kuriems apibrėžtos tikimybės jų trajektorijų tėkmei. Fizikiniai atsitiktinių procesų pavyzdžiai: skystyje pakibusios medžiagos dalelės judesiai, nulemti skysčio molekulių chaotiško šiluminio judėjimo (Browno judesio procesas) arba radioaktyviosios medžiagos atomų skilimų per tam tikrą laiko intervalą skaičius (Poissono procesas). Atsitiktinių procesų teorija nagrinėja metodus, kaip sukonstruoti atsitiktinius procesus su turimais baigtiniamačiais tikimybiniais skirstiniais ir ištirti jų trajektorijų savybes bei rasti įvairias skaitines charakteristikas. Greta funkcijų teorijos, funkcinės analizės, diferencialinių lygčių teorijos plačiai taikomi stochastinės analizės metodai, kuriems labai svarbios yra martingalo, atsitiktinio taškinio mato bei stochastinių integralų sąvokos. Atsitiktinis procesas X apibrėžiamas tikimybine erdve (Ω, , P), laiko aibe T ⊂ R1 ir būsenų išmatuojamąja erdve (, ), kaip atvaizdis X : Ω × T → , kuriam su visais tj ∈ T, Aj ∈ , j = 1, …, n, n ≥ 1, {ω : X(ω, tj) ∈ Aj, j = 1, …, n} ∈ . Kartu apibrėžiamos funkcijos (A1, …, An) = P({ω : X(ω, tj) ∈ Aj, j = 1, …, n}), vadinamos baigtiniamačiais proceso X tikimybiniais skirstiniais. Atsitiktiniai procesai klasifikuojami pagal rinkinio tarpusavyje suderintų baigtiniamačių tikimybinių skirstinių Ft1, …, tn (A1, …, An), tj ∈ T, Aj ∈ A, j = 1, …, n, n ≥ 1, turinčių tikimybių prasmę, jog laiko momentais tj proceso trajektorija patenka į būsenų poaibį Aj, j = 1, …, n, savybes. Atsitiktinių procesų svarbiausiosios klasės: Gausso, Markovo, stacionariųjų ir antrosios eilės procesų. Gausso procesų klasė apibūdinama tuo, kad visi jų baigtiniamačiai tikimybiniai skirstiniai yra Gausso skirstiniai. Jie visiškai aprašomi vidurkio ir koreliacijos funkcijomis. Markovo procesų klasė apibūdinama tuo, jog jų baigtiniamačiai tikimybiniai skirstiniai išreiškiami pradiniu skirstiniu F0(A) ir perėjimo tikimybių funkcija P(s, x; t, A) : F0, t1, …, tn (A0, A1, …, An) = F0, (dx0) P(0, x0; t1, dx1)… P(tn–1, xn–1; tn, dxn), 0 < t1 < … < tn, n ≥ 1. Stacionarieji procesai yra tokie atsitiktiniai procesai, kurių baigtiniamačiai tikimybiniai skirstiniai nepriklauso nuo laiko postūmio. Atsitiktiniai procesai vadinami antrosios eilės atsitiktiniais procesais, jei jų būsenų erdvė yra kompleksiniai skaičiai ir jų reikšmių modulio kvadratas kiekvienu laiko momentu turi baigtinį vidurkį. Atsitiktiniai procesai kaip matematiniai modeliai taikomi beveik visose mokslo ir technikos šakose. Atsitiktinių procesų teorija grindžiama A. Kolmogorovo, N. Wienerio, S. Bernšteino, A. Chinčino, P. Lévy, W. Fellerio, J. L. Doobo, K. Ito ir kitų matematikų darbais. Lietuvoje tiriami Markovo procesai bei jų optimalusis valdymas, praplėsta stochastinė analizė ir stochastinių diferencialinių lygčių teorija, sukonstruotos naujos stacionariųjų procesų klasės ir išnagrinėtos jų ergodinės savybės, tiriamos atsitiktinių procesų aproksimacijos.'

delfi_input = 'Statant tiltą ledonešio rizikos nenumatė Ar pasikartos prieš dešimtmetį įvykusi nelaimė, kai Neryje ties Kauno stichiškai susiformavus ledo lyčių sangrūdoms buvo užsemtas Radikių kaimas Kaune rajone? Ekspertams prognozuojant, kad šįkart potvynis gali vėl pasikartoti, Kauno rajono savivaldybė jau ruošiasi galimam stichijos šėlsmui. Ypač daug nerimo kelia tai, kad statant naująjį tiltą per Nerį, ženkliai susiaurinta upės vaga. Link Kauno tekančios Neries vanduo šioje vietoje patenka tarsi į piltuvą. Upė šiuo metu vis dar užšalusi, gali būti, jog ledai pajudės šios savaitės pabaigoje, ar kitą savaitę.Praėjusią savaitę Kauno rajono savivaldybėje surengtame Ekstremaliųjų situacijų operacijų centro posėdyje aiškintasi, ką reikėtų daryti, jeigu susiformuotų ledų sangrūdos ir grėstų potvynis. „Ledo lytys kai kuriose Neries vietose jau yra sustojusios ir tai kelia nerimą. Nesinorėtų, kad pasikartotų prieš 10 metų įvykusi situacija“, – sakė Kauno rajono savivaldybės administracijos direktoriaus pavaduotojas Mantas Rikteris.Paaiškėjo, kad ruošianti naujojo tilto, turėsiančio išspręsti spūsčių problemą kelyje Vilnius – Klaipėda problemą, nebuvo įvertinta ledonešio rizika, nenumatyta apsauga nuo galimo potvynio. Kauno rajone baiminamasi pavasarinio potvynio: statant tiltą per Nerį susiaurėjo upės vaga.automobilių kelių direkcijos Saugaus eismo skyriaus vedėjas Nemunas Abukauskas. „Projektuojant tokius objektus ir vykdant statybas nenumatome gamtinių sąlygų sukeliamų problemų. Manome, kad dėl statybų susiaurėjusi upės vaga neturėtų lemiamos įtakos galimam potvyniui. Žinoma, ekstremalioji situacija gali įvykti, todėl rangovas stebi teritoriją, nes jis taip pat suinteresuotas apsaugoti statybvietę“, – teigė N. Abukauskas. Ledą laužo žemkasė su kaušu Naująjį tiltą statančios bendrovės „Kauno tiltai“ atstovai teigia, jog rūpinasi galinčiomis kilti grėsmėmis. Pasamdyta specialią techniką turinti įmonė „Hidrum“, kad laužytų ledą tilto prieigose. Šiuo metu Neryje dirbanti daugiafunkcė žemkasė „Watermaster Classic IV“, į kurią dedamos nemažos viltys, gali dirbti tik tol, kol upėje nepajudėjo ledai. Prasidėjus ledonešiui šis laivas taps bejėgis prieš stichiją, darbus teks nutraukti. Tai portalui „Delfi“ sakė uždarosios akcinės bendrovės „Hidrum“ gamybos direktorius Darius Šapkauskas.„Kol kas laužyti ledą pavyksta, jo storis – 25 cm., prie kranto apie 40 cm. Laužomas ledas, kad upė turėtų prataką ir plaukiantys ledai galėtų nesustoję praplaukti pro tiltą“, – sakė D. Šapkauskas. Bendrovės „Hidrum“ atstovas atkreipė dėmesį, kad pavojų keliančios ledo sangrūdos gali susiformuoti ne ties statomu tiltu, o kitoje vietoje – už kelių šimtų metrų aukštyn upe esančios salos prieigose. „Mano manymu, pagrindinė ledų susikaupimo vieta bus prieš šią salą. Tai natūrali kliūtis, dėl to prieš 10 metų įvyko potvyniai, nes salose susigrūdę ledais sukėlė vandens lygį“, – sakė hidrotechnikas. Jis svarstė, kad viskas priklausys nuo oro temperatūros. Jeigu ji bus teigiama ir naktimis, ir dieną, potvynio tikimybė padidės. Jeigu naktimis šals, potvynio gali ir nebūti. „Dar vienas svarbus akcentas: pirmiausia žemę užsnigo storu sluoksniu, įšalo praktiškai nėra, todėl pats gruntas akumuliuos didelę vandens dalį. Tai viena priežasčių, kodėl gali nebūti didelio potvynio“, – sakė „Kauno tiltų“ pasamdytos bendrovės „Hidrum“ gamybos direktorius D. Šapkauskas. Situaciją stebi vaizdo kameromis Tiltą statančios bendrovės „Kauno tiltai“ atstovė spaudai Giedrė Samulionytė – Lekavičienė minėjo, kad ruošiantis galimam stichijos proveržiui ne tik pasamdyta įmonė ledams laužyti, bet ir situacija nuolatos stebima ir vaizdo kameromis. „Prasidėjus ledų sangrūdoms ties statomu tiltu, kaip ir nutarta Kauno rajono ekstremalių situacijų valdymo posėdyje, nedelsiant informuotume tarnybas ir, pagal galimybes, dirbtume su mūsų technika. Naujai statomas tiltas per Nerį draustas statybinių rizikų draudimu. Tokiu draudžiame ir visus kitus objektus.Norime pažymėti, kad AB „Kauno tiltai“ atsako tik už ties objektu esančios upės vagos situaciją. Ekspertų nuomone, pagrindinė ledų sangrūdos ir potvynio rizika yra aukštutinėje upės dalyje nuo statomo tilto vietos. Kaip ir minėta, dedamos maksimalios pastangos tiek atliekant ledonešio monitoringą, tiek aktyviai bendradarbiaujant su kitomis institucijomis siekiant užkirsti kelią potvynio grėsmei“, – sakė „Kauno tiltų“ atstovė G. Samulionytė – Lekavičienė. Žemkasė ledonešio neatlaikys – pajudėjus lytims darbus teks nutraukti Šiuo metu ledus laužanti žemkasė dirba tik dienomis. Pasak „Hidrum“ atstovo, jeigu reikės, speciali technika čia darbuosis ir visą parą. Ar nedidelis laivas galės pasipriešinti pajudėsiančiai ledo lyčių masei ? „Pajudėjus ledams upėje visu perimetru mes turėtume pasitraukti. Laivas – 17 tonų svorio, bet su ledų mase niekas dar nekariavo“, – sakė D. Šapkauskas. Užsiminus apie teorinę galimybę pasitelkti ledlaužį, „Hidrum“ gamybos direktorius sakė, kad jis padėtų nebent tik šiuo momentu – galėtų išlaužyti ledus ir atlaisvinti upės vagą. O prieš prasidėjusį ledonešį jis tiesiog būtų bejėgis. Vienas galimų variantų, kaip sumažinti potvynio pavojų – Nerį tebekaustantį ledą barstyti durpėmis. Šviečiant saulei tamsus jų paviršius greitai įkaista ir tirpina ledą. Prieš 10 metų nuo potvynio stipriai nukentėjusio Radikių kaimo bendruomenės pirmininkas Juozas Ravinis prognozavo, kad šiais metais stipresnio potvynio neturėtų būti – ledo sluoksnis upėje nėra storas. Jis apgailestavo, kad draudimo bendrovės itin nenoriai draudžia gyventojų turtą nuo potvynio žalos. Vadovaujamasi principu, kad įsikūrus žemesnėse vietose, prie upės žmogus tarsi pats prisiima riziką dėl galimo potvynio padarinių.Kauno rajono savivaldybė skelbia pavasarį Radikiuose pradėsianti apsauginio pylimo statybas. Ši apsaugos nuo potvynio priemonė kainuos 1,241 mln. eurų, savivaldybė tam skirs 93 tūkst. eurų, o likusią dalį sudarys Europos Sąjungos lėšos. Tačiau kaimo bendruomenės pirmininkas J. Ravinis abejoja, ar iškilsiantis pylimas padės efektyviai apsaugoti gyventojų namus nuo potvynio. Jis pastebėjo, kad potvynio vanduo galės patekti per į Nerį įtekantį upelį… Ledus planuojama ir tirpdyti, ir sprogdinti Ruošianti galimam potvyniui Kauno rajono savivaldybė kreipėsi į aplinkosaugininkus klausdama, ar būtų duotas leidimas sprogdinti ledus ir ar tai nepakenks gamtai. Taip pat nuspręsta atlikti rinkos vertinimą, kur būtų galima įsigyti durpių ir kokiu būdu būtų galima jas išbarstyti ant ledo, kad lytys imtų korėti ir ištirptų upėje. Į potvynio teritoriją galinčių patekti vietovių gyventojas jau platinami lankstinukai, kuriuose nurodoma, kaip elgtis stichinės nelaimės atveju. Žmonėms aiškinama ne tik kokias atsargas sukaupti, bet ir kaip išsikviesti pagalbą. Žmonės raginami evakuotis, išmokti pagalbos ženklus „Priešgaisrinės apsaugos ir gelbėjimo departamentas galimai užliejamose teritorijose gyvenančioms nėščiosioms, ligoniams, senyvo amžiaus žmonėms ir vaikams rekomenduoja išvykti į saugesnes vietas. Nusprendusiems likti potvynio zonoje, patariama susipažinti su parengta gyvenamosios teritorijos evakuacijos schema, kurią galite rasti seniūnijų skelbimo lentose. Gyventojams rekomenduojama apsirūpinti neperšlampama apranga, ilgai negendančiais maisto produktais, medikamentais, mobiliojo ryšio priemonėmis ir kitais būtinais daiktais. Būtina izoliuoti visus elektros lizdus. Esant apsėmimo pavojui iš rūsių išneškite maisto atsargas, vertingus daiktus. Išvežkite trąšų ir pesticidų atsargas, nepalikite atvirų teršalų, kurie gali pakenkti aplinkai. Jei nutarėte išvykti į saugesnę vietą, prieš palikdami namus užsukite vandens, dujų sklendes, išjunkite elektrą, pasiimkite dokumentus, būtiniausius daiktus. Kad būtų žinoma, kiek žmonių liko užtvindytoje teritorijoje, išvykdami užsiregistruokite seniūnijoje. Palaikykite ryšį su kaimynais, būkite pasiruošę vieni kitiems padėti. Mokėkite pagalbos ženklus: iškeltas audeklo gabalas (naktį – žibinto šviesa) reiškia, kad reikalinga pagalba, mosavimas audeklo gabalu (naktį – mirksinti žibinto šviesa) – reikalinga skubi pagalba”, – skelbiama Kauno rajono gyventojams platinamuose pranešimuose.'
min_input = 'Susitelkti teko visiems Praeitos savaitės pabaigoje progimnazijos Tėvų tarybos pirmininkas Arūnas Penkaitis sakė, kad grįžimo į mokyklą laukiama kaip šventės, kaip gimtadienio: „Anksčiau, kai išties turėjome tam tikrą informacijos trūkumą, nuotaikų buvo įvairių. Šiuo metu, gavus atsakymus į visus kilusius klausimus, didžioji dauguma bendruomenės nusiteikusi labai pozityviai.“ Mokyklos direktorius R.Remeika praėjusią savaitę 15min teigė, kad nuotolinis ugdymas nėra toks efektyvus, todėl siekiama kuo greičiau pereiti į kontaktinį mokymą. Pirmadienio rytą jis feisbuke džiaugėsi, kad bendruomenė susitelkė ir atsakingai dirbdama parodė, kiek daug vaikų labui gali padaryti. O ši savaitė taip pat būsianti susitelkimo metas, kai teks griežtai laikytis visų saugumo taisyklių tiek mokykloje, tiek namuose, tiek viešoje erdvėje, nes esą nėra „prabangos“ susirgti ir taip rizikuoti aplinkinių sveikata. Rezultatus vertins po dar vienų testų Anot Vilniaus mero Remigijaus Šimašiaus, prieš grįžtant į klases buvo testuojami ir darbuotojai, ir mokiniai, ir jų šeimų nariai, tad iššūkių netrūko. „Savaitės pabaigoje vėl bus atliekami testai, žiūrėsime, kokie rezultatai. Nes patys suprantame, kad švietimas yra ta sritis, kur labai norisi grįžti į normalias vėžes ir dar geresnes – per pandemiją išmoko naujų dalykų ir mokytojai, ir mokiniai. Kartu suprantame tą riziką – iš Estijos gavome ataskaitą, kad mokyklose didelė rizika, vaikai vieni kitiems perneša. Mūsų tą eksperimentą Lietuvos mastu traktuoju kaip labai svarbų – kaip organizuoti ir kaip paskui viskas vyks“, – kalbėjo R.Šimašius. Mero teigimu, jeigu reikėtų pagal tokią pačią procedūrą atidaryti visas Lietuvos ar Vilniaus mokyklas, tai greičiausia nebūtų įmanoma – resursų niekam ir niekada neužtektų. Todėl reikia apibendrinti išvadas, kaip tai galima padaryti paprasčiau. Apie tai ketinama diskutuoti ir su Vyriausybe. Atsidarytų nors rytoj Paklaustas, kada galėtų visos mokyklos atsidaryti, meras sakė, kad jeigu virusas iš Lietuvos išeitų, tai galima būtų ir rytoj padaryti. „Bet kada jis išeis, kokiu tempu, kokios variacijos, kiti dalykai, negaliu pasakyti. Bet jeigu epidemiologinė situacija leistų, sugrįžti į mokyklas mums nebūtų problemų. Nebent kiltų vienai kitai šeimai, nes ne paslaptis, kad ne vienas moksleivis, kartais ir mokytojai pamokose dalyvauja ne iš Vilniaus, kartais net ir į užsienį išvažiavę“, – sakė R.Šimašius. Jis įsitikinęs, kad tokių ar panašių eksperimentų turėtų būti ir daugiau, tai leistų lengviau grįžti visiems, kai epidemiologinė situacija leis. Vilnius rengė konferenciją su mokytojais, svarstė nuotolinio mokymosi sąlygas ir būdus, tai joje paaiškėjo, kad tiek abiturientai, tiek mokytojai vieningai sako, kad egzaminų vėlinti negalima ir nereikia, yra pasiūlymų, kaip organizuoti juos tinkamai įprastu metu. Buvo teigiamų testų Vilniaus visuomenės sveikatos biuro vadovė Guoda Ropaitė-Beigė teigė, kad viso šio proceso metu mokyklos bendruomenė buvo itin aktyvi: „Ketvirtadienį startavome su pedagogų testavimu kaupinių principu, tai 93 asmenys, sudarėme 39 kaupinius, visi buvo neigiami. Tai tikrai mokytojai saugūs ir sveiki grįžo į šitą įstaigą. O kalbant apie namų ūkius – t. y. vaikus ir jų tėvelius, testavimas vyko penktadienį, šeštadienį ir dar sudarėme galimybę ir sekmadienį testuotis. Buvo per 500 kaupinių, daugiau nei 2 tūkst. asmenų atvyko testuotis į testavimo punktą, keturi kaupiniai buvo teigiami, dabar vykdomi tyrimai PGR tyrimai.“ Šis testavimo būdas skiriasi nuo PGR tuo, kad pastarojo atveju imamas mėginys iš nosiaryklės ir ryklės, o kaupinio – iš nosies landų ir tie ėminiai grupuojami: vienoje virusologinėje terpėje grupuojami ir dedami kelių asmenų ėminius ir jie tiriami. Jeigu gaunamas teigiamas rezultatas, tuomet jau tiems asmenims atliekami PGR testai. Bandomojo projekto laikas – dvi savaitės, pirmoji jau praėjo, antroji yra ši, tad jai baigiantis, penktadienį, planuojamas vizitas į mokyklą, ten bus tiriamos klasės, mėginama pasižiūrėti, kokia situacija savaitę pasimokius kontaktiniu būdu. Jeigu Vyriausybė priims sprendimus, tada bus mokomasi ir kitą savaitę. Šią savaitę Vilnius turėtų gauti 11 tūkst. vakcinų nuo COVID-19, tai džiugina, anot mero, tačiau kiekis vis dar yra per mažas, kad galima būtų sakyti, jog vakcina buvo įveikta pandemija. '

test = bertSummarize(delfi_input)
test

'Tai portalui „Delfi“ sakė uždarosios akcinės bendrovės „Hidrum“ gamybos direktorius Darius Šapkauskas.„Kol kas laužyti ledą pavyksta, jo storis – 25 cm., prie kranto apie 40 cm. Situaciją stebi vaizdo kameromis Tiltą statančios bendrovės „Kauno tiltai“ atstovė spaudai Giedrė Samulionytė – Lekavičienė minėjo, kad ruošiantis galimam stichijos proveržiui ne tik pasamdyta įmonė ledams laužyti, bet ir situacija nuolatos stebima ir vaizdo kameromis. „Prasidėjus ledų sangrūdoms ties statomu tiltu, kaip ir nutarta Kauno rajono ekstremalių situacijų valdymo posėdyje, nedelsiant informuotume tarnybas ir, pagal galimybes, dirbtume su mūsų technika. Naujai statomas tiltas per Nerį draustas statybinių rizikų draudimu. Prieš 10 metų nuo potvynio stipriai nukentėjusio Radikių kaimo bendruomenės pirmininkas Juozas Ravinis prognozavo, kad šiais metais stipresnio potvynio neturėtų būti – ledo sluoksnis upėje nėra storas. Vadovaujamasi principu, kad įsikūrus žemesnėse vietose, prie upės žmogus tarsi pat

### Evaluate

In [372]:
from rouge import Rouge
rouge = Rouge()

In [373]:
tikroji_delfi_santrauka = 'Atšilus orams, tirpstant sniegui, o Nerį vis dar kaustant ledui Kauno rajone baiminamasi potvynio. Tikimybę, kad ties Kleboniškio mišku gali susiformuoti potvynį sukelsiančios ledo lyčių sangrūdos šįkart didina tebevykstančios naujo tilto statybos, jų metu ženkliai susiaurinta upės vaga. Didžiulio potvynio būta 2010 metais, tuomet upės lygiui staiga pakilus keliais metrais nukentėjo Kauno rajone esanti Radikių kaimas, buvo užsemti ir kai kurie Kleboniškio miške iškilę namai.'

In [374]:
rouge.get_scores(tikroji_delfi_santrauka, test)

[{'rouge-1': {'f': 0.09361701727478515,
   'p': 0.16923076923076924,
   'r': 0.06470588235294118},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.08411214549742355, 'p': 0.15, 'r': 0.05844155844155844}}]